In [1]:
import dash
from dash import html, dcc
import pandas as pd
import plotly.express as px

# Load and prepare data
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url)
df.columns = [col.lower().replace(' ', '_') for col in df.columns]
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], errors='coerce')
df['month'] = df['effective_to_date'].dt.month_name()
df['number_of_open_complaints'] = pd.to_numeric(df['number_of_open_complaints'], errors='coerce').fillna(0).astype(int)

# Filter for complaints
complaints = df[df['number_of_open_complaints'] > 0]
complaints_by_policy_month = complaints.groupby(['policy_type', 'month'])['number_of_open_complaints'].count().reset_index()
complaints_by_policy_month.rename(columns={'number_of_open_complaints': 'complaint_count'}, inplace=True)

# Create bar chart
fig = px.bar(
    complaints_by_policy_month,
    x='month',
    y='complaint_count',
    color='policy_type',
    barmode='group',
    title='Number of Complaints by Policy Type and Month'
)

# Build app layout
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Customer Complaints Dashboard"),
    dcc.Graph(figure=fig)
])

if __name__ == '__main__':
    app.run(debug=True)